# Adversarial Simulator for an online endpont

## Objective

This tutorial provides a step-by-step guide on how to leverage adversarial simulator to simulate an adversarial question answering scenario against an online endpoint

This tutorial uses the following Azure AI services:

- [Azure AI Safety Evaluation](https://aka.ms/azureaistudiosafetyeval)
- [azure-ai-evaluation](https://learn.microsoft.com/en-us/azure/ai-studio/how-to/develop/evaluate-sdk)

## Time

You should expect to spend 20 minutes running this sample. 

## About this example

This example demonstrates a simulated adversarial question answering. It is important to have access to AzureOpenAI credentials and an AzureAI project.

## Before you begin
### Prerequesite
[Have an online deployment on Azure AI studio](https://learn.microsoft.com/en-us/azure/machine-learning/concept-endpoints-online?view=azureml-api-2)
### Installation

Install the following packages required to execute this notebook. 


In [1]:
%pip install azure-ai-evaluation

Note: you may need to restart the kernel to use updated packages.


### Parameters and imports

In [2]:
from pathlib import Path
from azure.ai.evaluation.simulator import AdversarialSimulator, AdversarialScenario
from typing import Optional, List, Dict, Any
import os
from openai import AzureOpenAI

## Target function
The target function for this sample uses a call to call the endpoint.

Make sure you retrive the `api_key`, `endpoint` and `azure_model_deployment` from Azure AI studio and update them in the `call_endpoint` function below. 

In [3]:
azure_ai_project = {
    "subscription_id": "<your-subscription-id>",
    "resource_group_name": "<your-resource-group-name>",
    "project_name": "<your-project-name>",
}

In [5]:
# Use the following code to set the environment variables if not already set. If set, you can skip this step.

os.environ["AZURE_OPENAI_API_KEY"] = "<your-api-key>"
os.environ["AZURE_OPENAI_API_VERSION"] = "<api version>"
os.environ["AZURE_OPENAI_DEPLOYMENT"] = "<your-deployment>"
os.environ["AZURE_OPENAI_ENDPOINT"] = "<your-endpoint>"

In [7]:
def call_endpoint(query: str) -> dict:
    deployment = os.environ.get("AZURE_OPENAI_DEPLOYMENT")
    endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
    # Get a client handle for the model
    client = AzureOpenAI(
        azure_endpoint=endpoint,
        api_version=os.environ.get("AZURE_OPENAI_API_VERSION"),
        api_key=os.environ.get("AZURE_OPENAI_API_KEY"),
    )
    # Call the model
    completion = client.chat.completions.create(
        model=deployment,
        messages=[
            {
                "role": "user",
                "content": query,
            }
        ],
        max_tokens=800,
        temperature=0.7,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None,
        stream=False,
    )
    return completion.to_dict()

## Initialize the simulator

In [8]:
simulator = AdversarialSimulator(azure_ai_project=azure_ai_project)

### Run the simulator

The interactions between your application (in this case, ask_wiki) and the adversarial simulator is managed by a callback method and this method is used to format the request to your application and the response from the application.

In [12]:
## define a callback that formats the interaction between the simulator and the online endpoint


async def callback(
    messages: List[Dict],
    stream: bool = False,
    session_state: Any = None,  # noqa: ANN401
    context: Optional[Dict[str, Any]] = None,
) -> dict:
    messages_list = messages["messages"]
    query = messages_list[-1]["content"]
    context = None
    try:
        response = call_endpoint(query)
        # we are formatting the response to follow the openAI chat protocol format
        formatted_response = {
            "content": response["choices"][0]["message"]["content"],
            "role": "assistant",
            "context": {context},
        }
    except Exception as e:
        response = f"Something went wrong {str(e)}"
        formatted_response = None
    messages["messages"].append(formatted_response)
    return {"messages": messages_list, "stream": stream, "session_state": session_state, "context": context}

In [13]:
outputs = await simulator(
    scenario=AdversarialScenario.ADVERSARIAL_QA, max_conversation_turns=1, max_simulation_results=1, target=callback
)

generating simulations: 100%|████████████████████████████████| 1/1 [00:01<00:00,  1.75s/simulations]


### Convert the outputs to a format that can be evaluated

In [11]:
with Path.open("outputs.jsonl", "w") as f:
    f.write(outputs.to_eval_qa_json_lines())